# 7 Total Possible Groups of Users

1. [Users that received offers](#r)
    1. [Users that viewed offers](#rv)
        1. [Users that completed offers](#rvc)
        2. [Users that did not complete offers, but still had transactions](#rvt)
        3. [Users that did not have transactions](#rvn)
    2. [Users that did not view offers](#rn)
        1. [Users that completed offers](#rnc)
        2. [Users that did not complete offers, but still had transactions](#rnt)
        3. [Users that did not have transactions](#rnn)
2. [Users that did not have any offers, only transactions](#only_transactions)
    - We don't really care about these users since they did not have an offer to respond to to begin with

# Library Imports

In [99]:
import pandas as pd

from sqlalchemy import create_engine

# Loading Cleaned Data

In [100]:
conn = create_engine('sqlite:///data/starbucks_data.db')

offers = pd.read_sql('SELECT * FROM offers', conn)
offers_wo_chnl = offers.drop(columns=['offer_notification_channel', 'offer_type', 'offer_spend_minimum', 'offer_reward']).drop_duplicates()

interactions = pd.read_sql('SELECT * FROM interactions', conn)

## Separating Out Interactions Data
Here I am separating the user interactions by their event type. These event types are for noting if the user has __received__, __viewed__, or __completed__ an offer. Additionally it notes what __transactions__ the user had.

When receiving an offer, the offer has a duration assigned. The user must complete the offer within this duration in order for it to count as completed.

In [101]:
base_col_order = ['user_id', 'offer_id', 'intxn_event_type', 'intxn_time', 'intxn_amount', 'intxn_reward']

offer_received = interactions[ interactions['intxn_event_type'] == 'offer received' ].copy().drop(['intxn_amount', 'intxn_reward'], axis=1)
offer_received_cols = [ "received" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in offer_received.columns ]
offer_received.columns = offer_received_cols

offer_received = pd.merge(offer_received, offers_wo_chnl, how='left', on='offer_id')
offer_received['expiration_time'] = offer_received['received_time'] + offer_received['offer_duration']
offer_received = offer_received.drop(columns=['offer_duration'])

offer_viewed = interactions[ interactions['intxn_event_type'] == 'offer viewed' ].copy().drop(['intxn_amount', 'intxn_reward'], axis=1)
offer_viewed_cols = [ "viewed" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in offer_viewed.columns ]
offer_viewed.columns = offer_viewed_cols

offer_completed = interactions[ interactions['intxn_event_type'] == 'offer completed' ].copy().drop(['intxn_amount', 'intxn_reward'], axis=1)
offer_completed_cols = [ "completed" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in offer_completed.columns ]
offer_completed.columns = offer_completed_cols

transactions = interactions[ interactions['intxn_event_type'] == 'transaction' ].copy().drop(['offer_id', 'intxn_reward'], axis=1)
transactions_cols = [ "transaction" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in transactions.columns ]
transactions.columns = transactions_cols

<a id='r'></a>
# Users that received offers

In [102]:
offer_received.head()

,user_id,offer_id,received_event_type,received_time,offer_reward_vs_spend,expiration_time
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,0,1.00,168
1,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,0,0.25,240
2,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168
3,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,offer received,0,0.20,240
4,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0,offer received,0,1.00,120


<a id='rv'></a>
## Users that viewed offers

In [103]:
offer_recv_view = pd.merge( offer_received, offer_viewed, on=['user_id', 'offer_id'], how='left' )
offer_recv_view = offer_recv_view[ ~offer_recv_view['viewed_time'].isna() ]
# Removing duplicates caused by receiving the same offer multiple times and the join including those received offers before the viewed ones
offer_recv_view = offer_recv_view[ offer_recv_view['received_time'] <= offer_recv_view['viewed_time'] ]
offer_recv_view = offer_recv_view[ offer_recv_view['viewed_time'] <= offer_recv_view['expiration_time'] ]
# Removing duplicates caused by viewing the same offer after receiving it another time
offer_recv_view = offer_recv_view.sort_values(['user_id', 'received_time', 'expiration_time', 'viewed_time']).drop_duplicates(subset=['offer_id', 'user_id', 'received_time', 'expiration_time'], keep='first')

offer_recv_view

,user_id,offer_id,received_event_type,received_time,offer_reward_vs_spend,expiration_time,viewed_event_type,viewed_time
19437,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,offer received,168,0.00,240,offer viewed,192.0
35295,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,offer received,336,0.00,432,offer viewed,372.0
51166,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,offer received,408,1.00,528,offer viewed,456.0
66965,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,offer received,504,0.20,744,offer viewed,540.0
20066,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,offer received,168,1.00,288,offer viewed,216.0
...,...,...,...,...,...,...,...,...
17694,ffff82501cea40309d5fdd7edcca4a07,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,168,0.25,408,offer viewed,174.0
33595,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,offer received,336,0.20,504,offer viewed,354.0
49455,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,offer received,408,0.20,576,offer viewed,414.0
65289,ffff82501cea40309d5fdd7edcca4a07,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,504,1.00,672,offer viewed,534.0


<a id='rvc'></a>
### Users that completed the offer

In [104]:
offer_recv_view_comp = pd.merge( offer_recv_view, offer_completed, on=['user_id', 'offer_id'], how='left' )
offer_recv_view_comp = offer_recv_view_comp[ ~offer_recv_view_comp['completed_time'].isna() ]
# Removing duplicates caused by completing the same offer before receiving it another time
offer_recv_view_comp = offer_recv_view_comp[ offer_recv_view_comp['completed_time'] >= offer_recv_view_comp['received_time'] ]
# Removing duplicates caused by completing the same offer after viewing it another time
offer_recv_view_comp = offer_recv_view_comp[ offer_recv_view_comp['completed_time'] >= offer_recv_view_comp['viewed_time'] ]
offer_recv_view_comp = offer_recv_view_comp[ offer_recv_view_comp['completed_time'] <= offer_recv_view_comp['expiration_time'] ]
# Removing duplicates caused by viewing the same offer after receiving it another time
offer_recv_view_comp = offer_recv_view_comp.sort_values(['user_id', 'received_time', 'viewed_time', 'completed_time']).drop_duplicates(subset=['offer_id', 'user_id', 'received_time', 'viewed_time'], keep='first')

offer_recv_view_comp

,user_id,offer_id,received_event_type,received_time,offer_reward_vs_spend,expiration_time,viewed_event_type,viewed_time,completed_event_type,completed_time
7,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,offer received,168,0.43,336,offer viewed,186.0,offer completed,252.0
9,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,408,0.25,648,offer viewed,432.0,offer completed,576.0
10,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,504,1.00,672,offer viewed,516.0,offer completed,576.0
11,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,offer received,0,0.20,240,offer viewed,12.0,offer completed,54.0
13,0020c2b971eb4e9188eac86d93036a77,4d5c57ea9a6940dd891ad53e9dbe8da0,offer received,408,1.00,528,offer viewed,426.0,offer completed,510.0
...,...,...,...,...,...,...,...,...,...,...
65565,fffad4f4828548d1b5583907f2e9906b,f19421c1d4aa40978ebb69ca19b0e20d,offer received,408,1.00,528,offer viewed,510.0,offer completed,516.0
65567,ffff82501cea40309d5fdd7edcca4a07,fafdcd668e3743c1bb461111dcafc2a4,offer received,0,0.20,240,offer viewed,6.0,offer completed,60.0
65568,ffff82501cea40309d5fdd7edcca4a07,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,168,0.25,408,offer viewed,174.0,offer completed,198.0
65569,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,offer received,336,0.20,504,offer viewed,354.0,offer completed,384.0


In [105]:
offer_recv_view_comp.to_sql('offer_recv_view_comp', conn, if_exists='replace', index=False)

23864

<a id='rvt'></a>
### Users that did not complete the offer

In [106]:
offer_recv_view_nocomp = pd.merge( offer_recv_view, offer_completed, on=['user_id', 'offer_id'], how='left' )
offer_recv_view_nocomp = offer_recv_view_nocomp[ offer_recv_view_nocomp['completed_time'].isna() ]
offer_recv_view_nocomp = pd.merge( offer_recv_view_nocomp, transactions, on=['user_id'], how='left' )
offer_recv_view_nocomp = offer_recv_view_nocomp[ offer_recv_view_nocomp['transaction_time'] > offer_recv_view_nocomp['received_time'] ]
offer_recv_view_nocomp = offer_recv_view_nocomp[ offer_recv_view_nocomp['transaction_time'] > offer_recv_view_nocomp['viewed_time'] ]
offer_recv_view_nocomp = offer_recv_view_nocomp[ offer_recv_view_nocomp['transaction_time'] < offer_recv_view_nocomp['expiration_time'] ]
offer_recv_view_nocomp_agg = offer_recv_view_nocomp.groupby(['user_id', 'offer_id', 'received_event_type', 'viewed_event_type']).agg(
    offer_received_time=('received_time', 'first'),
    offer_expiration_time=('expiration_time', 'first'),
    offer_viewed_time=('viewed_time', 'first'),
    transaction_time=('transaction_time', 'last'),
    transaction_count=('transaction_time', 'count'),
    transaction_sum=('transaction_amount', 'sum')
).reset_index()

offer_recv_view_nocomp_agg

,user_id,offer_id,received_event_type,viewed_event_type,offer_received_time,offer_expiration_time,offer_viewed_time,transaction_time,transaction_count,transaction_sum
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,offer received,offer viewed,336,432,372.0,414.0,1,8.57
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,offer received,offer viewed,168,240,192.0,228.0,1,22.16
2,0020ccbbb6d84e358d3414a3ff76cffd,5a8bc65990b245e5a138643cd4eb9837,offer received,offer viewed,408,480,408.0,462.0,4,58.87
3,003d66b6608740288d6cc97a6903f4f0,3f207df678b143eea3cee63160fa8bed,offer received,offer viewed,336,432,372.0,402.0,2,9.60
4,003d66b6608740288d6cc97a6903f4f0,5a8bc65990b245e5a138643cd4eb9837,offer received,offer viewed,0,72,36.0,66.0,1,0.44
...,...,...,...,...,...,...,...,...,...,...
13581,fff3ba4757bd42088c044ca26d73817a,5a8bc65990b245e5a138643cd4eb9837,offer received,offer viewed,504,576,540.0,552.0,1,26.09
13582,fff7576017104bcc8677a8d63322b5e1,4d5c57ea9a6940dd891ad53e9dbe8da0,offer received,offer viewed,168,288,180.0,276.0,2,10.50
13583,fff7576017104bcc8677a8d63322b5e1,ae264e3637204a6fb9bb56bc8210ddfd,offer received,offer viewed,408,576,414.0,558.0,1,2.08
13584,fff8957ea8b240a6b5e634b6ee8eafcf,fafdcd668e3743c1bb461111dcafc2a4,offer received,offer viewed,408,648,432.0,576.0,3,8.09


In [107]:
offer_recv_view_nocomp.to_sql('offer_recv_view_nocomp', conn, if_exists='replace', index=False)

30655

<a id='rvn'></a>
### Users that did not have transactions

In [108]:
offer_recv_view_notrans = pd.merge( offer_recv_view, transactions, on=['user_id'], how='left' )
offer_recv_view_notrans = offer_recv_view_notrans[ offer_recv_view_notrans['transaction_time'].isna() ]
offer_recv_view_notrans = offer_recv_view_notrans[ offer_recv_view_notrans['viewed_time'] < offer_recv_view_notrans['expiration_time'] ]

offer_recv_view_notrans

,user_id,offer_id,received_event_type,received_time,offer_reward_vs_spend,expiration_time,viewed_event_type,viewed_time,transaction_event_type,transaction_time,transaction_amount
3123,01d7da27b8934ba1b3602a0153e4415f,5a8bc65990b245e5a138643cd4eb9837,offer received,168,0.0,240,offer viewed,168.0,NaN,NaN,NaN
3124,01d7da27b8934ba1b3602a0153e4415f,fafdcd668e3743c1bb461111dcafc2a4,offer received,336,0.2,576,offer viewed,336.0,NaN,NaN,NaN
3125,01d7da27b8934ba1b3602a0153e4415f,5a8bc65990b245e5a138643cd4eb9837,offer received,408,0.0,480,offer viewed,414.0,NaN,NaN,NaN
3126,01d7da27b8934ba1b3602a0153e4415f,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,576,1.0,744,offer viewed,666.0,NaN,NaN,NaN
5524,031387fa9cfd47e88f4e4b4b4cb71925,fafdcd668e3743c1bb461111dcafc2a4,offer received,0,0.2,240,offer viewed,6.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
489414,ffd810f301ad42d6bfe98c4ba6ee4125,f19421c1d4aa40978ebb69ca19b0e20d,offer received,0,1.0,120,offer viewed,12.0,NaN,NaN,NaN
489415,ffd810f301ad42d6bfe98c4ba6ee4125,3f207df678b143eea3cee63160fa8bed,offer received,336,0.0,432,offer viewed,342.0,NaN,NaN,NaN
489416,ffd810f301ad42d6bfe98c4ba6ee4125,2906b810c7d4411798c6938adc9daaa5,offer received,408,0.2,576,offer viewed,492.0,NaN,NaN,NaN
489517,ffed75d3abc64b488982f50ed12878b5,4d5c57ea9a6940dd891ad53e9dbe8da0,offer received,0,1.0,120,offer viewed,18.0,NaN,NaN,NaN


<a id='rn'></a>
## Users that did not view offers

Don't care about being specific about these users. Just need to know that they did not respond to the offer sent to them

In [109]:
offer_recv_noview = pd.merge( offer_received, offer_viewed, on=['user_id', 'offer_id'], how='left' )
offer_recv_noview = offer_recv_noview[ offer_recv_noview['viewed_time'].isna() ]

display(offer_recv_noview)

,user_id,offer_id,received_event_type,received_time,offer_reward_vs_spend,expiration_time,viewed_event_type,viewed_time
11,2eeac8d8feae4a8cad5a6af0499a211d,3f207df678b143eea3cee63160fa8bed,offer received,0,0.00,96,NaN,NaN
26,65aba5c617294649aeb624da249e1ee5,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN
29,868317b9be554cb18e50bc68484749a2,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN
32,4beeb3ed64dd4898b0edf2f6b67426d3,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN
34,25c906289d154b66bf579693f89481c9,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN
...,...,...,...,...,...,...,...,...
95306,97ee6e7a12fe4064b260fa48bdd0330f,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,576,1.00,744,NaN,NaN
95307,bf3069b178fe40d789dd027901bc406b,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,576,0.25,816,NaN,NaN
95310,f653cf2d8bba42d0a53c2937ee2e5893,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,576,1.00,744,NaN,NaN
95317,cb23b66c56f64b109d673d5e56574529,2906b810c7d4411798c6938adc9daaa5,offer received,576,0.20,744,NaN,NaN


<a id='rnc'></a>
### Users that completed the offer

In [110]:
offer_recv_noview_comp = pd.merge( offer_recv_noview, offer_completed, on=['user_id', 'offer_id'], how='left' )
offer_recv_noview_comp = offer_recv_noview_comp[ ~offer_recv_noview_comp['completed_time'].isna() ]
offer_recv_noview_comp = offer_recv_noview_comp[ offer_recv_noview_comp['completed_time'] < offer_recv_noview_comp['expiration_time'] ]

offer_recv_noview_comp

,user_id,offer_id,received_event_type,received_time,offer_reward_vs_spend,expiration_time,viewed_event_type,viewed_time,completed_event_type,completed_time
2,868317b9be554cb18e50bc68484749a2,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN,offer completed,132.0
3,4beeb3ed64dd4898b0edf2f6b67426d3,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN,offer completed,120.0
13,88baa20c29a94178a43a7d68e5f039d4,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,0,0.25,240,NaN,NaN,offer completed,132.0
18,2481f1fcfbcb4b288e5a03af02d95373,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN,offer completed,0.0
25,bf2c086d4c4049289df1e0c8ea08f55c,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,0,1.00,168,NaN,NaN,offer completed,156.0
...,...,...,...,...,...,...,...,...,...,...
16658,3b34800e101b4f6d90ec2b2f8f969013,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,576,1.00,744,NaN,NaN,offer completed,654.0
16659,ef205cec5dd44fa19e7f0fad2b8a2e2b,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,576,1.00,744,NaN,NaN,offer completed,582.0
16662,4b02006a03994e368ec5e3ac0bcccfe1,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,576,1.00,744,NaN,NaN,offer completed,582.0
16673,f653cf2d8bba42d0a53c2937ee2e5893,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,576,1.00,744,NaN,NaN,offer completed,630.0


<a id='rnv'></a>
### Users that did not complete the offer

In [111]:
offer_recv_noview_nocomp = pd.merge( offer_recv_noview, offer_completed, on=['user_id', 'offer_id'], how='left' )
offer_recv_noview_nocomp = offer_recv_noview_nocomp[ offer_recv_noview_nocomp['completed_event_type'].isna() ]
offer_recv_noview_nocomp = pd.merge( offer_recv_noview_nocomp, transactions, on=['user_id'], how='left' )
offer_recv_noview_nocomp = offer_recv_noview_nocomp[ offer_recv_noview_nocomp['received_time'] < offer_recv_noview_nocomp['transaction_time'] ]
offer_recv_noview_nocomp = offer_recv_noview_nocomp[ offer_recv_noview_nocomp['transaction_time'] < offer_recv_noview_nocomp['expiration_time'] ]

display(offer_recv_noview_nocomp)

,user_id,offer_id,received_event_type,received_time,offer_reward_vs_spend,expiration_time,viewed_event_type,viewed_time,completed_event_type,completed_time,transaction_event_type,transaction_time,transaction_amount
4,65aba5c617294649aeb624da249e1ee5,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN,NaN,NaN,transaction,66.0,0.74
5,65aba5c617294649aeb624da249e1ee5,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN,NaN,NaN,transaction,144.0,2.73
33,39dbcf43e24d41f4bbf0f134157e0e1e,ae264e3637204a6fb9bb56bc8210ddfd,offer received,0,1.00,168,NaN,NaN,NaN,NaN,transaction,90.0,7.59
53,488773cae11f4b71875bfccd4e3e99f2,2906b810c7d4411798c6938adc9daaa5,offer received,0,0.20,168,NaN,NaN,NaN,NaN,transaction,30.0,1.69
64,1827da57bb924833a6f73c79b5f2bbc5,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,0,1.00,168,NaN,NaN,NaN,NaN,transaction,18.0,2.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76229,ff737c250d2343729ade04c4f6eb1001,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,576,0.25,816,NaN,NaN,NaN,NaN,transaction,624.0,0.29
76230,ff737c250d2343729ade04c4f6eb1001,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,576,0.25,816,NaN,NaN,NaN,NaN,transaction,666.0,4.27
76231,ff737c250d2343729ade04c4f6eb1001,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,576,0.25,816,NaN,NaN,NaN,NaN,transaction,690.0,2.43
76235,97ee6e7a12fe4064b260fa48bdd0330f,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,576,1.00,744,NaN,NaN,NaN,NaN,transaction,702.0,4.39


<a id='rnn'></a>
### Users that did not have transactions

In [112]:
offer_recv_noview_notrans = pd.merge( offer_recv_noview, transactions, on=['user_id'], how='left' )
offer_recv_noview_notrans = offer_recv_noview_notrans[ offer_recv_noview_notrans['transaction_time'].isna() ]
offer_recv_noview_notrans = offer_recv_noview_notrans[ offer_recv_noview_notrans['viewed_time'] < offer_recv_noview_notrans['expiration_time'] ]

offer_recv_noview_notrans

,user_id,offer_id,received_event_type,received_time,offer_reward_vs_spend,expiration_time,viewed_event_type,viewed_time,transaction_event_type,transaction_time,transaction_amount


<a id='only_transactions'></a>
# Users with only Transactions
Not necessary for our evaluation

In [113]:
transactions_no_promos = pd.merge( transactions, offer_received, on='user_id', how='left' )
transactions_no_promos = transactions_no_promos[ transactions_no_promos['offer_id'].isna() ]

transactions_no_promos

,user_id,transaction_event_type,transaction_time,transaction_amount,offer_id,received_event_type,received_time,offer_reward_vs_spend,expiration_time
4096,eb540099db834cf59001f83a4561aef3,transaction,6,4.74,NaN,NaN,NaN,NaN,NaN
14148,12ede229379747bd8d74ccdc20097ca3,transaction,18,16.62,NaN,NaN,NaN,NaN,NaN
124269,c6e579c6821c41d1a7a6a9cf936e91bb,transaction,174,0.65,NaN,NaN,NaN,NaN,NaN
166666,c6e579c6821c41d1a7a6a9cf936e91bb,transaction,222,1.91,NaN,NaN,NaN,NaN,NaN
238452,ae8111e7e8cd4b60a8d35c42c1110555,transaction,306,17.51,NaN,NaN,NaN,NaN,NaN
244267,eb540099db834cf59001f83a4561aef3,transaction,318,5.09,NaN,NaN,NaN,NaN,NaN
248661,3a4874d8f0ef42b9a1b72294902afea9,transaction,324,22.75,NaN,NaN,NaN,NaN,NaN
250562,12ede229379747bd8d74ccdc20097ca3,transaction,324,27.60,NaN,NaN,NaN,NaN,NaN
254212,ae8111e7e8cd4b60a8d35c42c1110555,transaction,330,28.70,NaN,NaN,NaN,NaN,NaN
294458,3a4874d8f0ef42b9a1b72294902afea9,transaction,378,27.20,NaN,NaN,NaN,NaN,NaN
